In [1]:
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn import datasets, linear_model
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge, SGDRegressor
from sklearn.pipeline import make_pipeline

DATASET_PATH = "datasets/"

def load_song_data(dataset_path=DATASET_PATH):
    df = pd.read_csv(dataset_path + 'year-prediction-msd-train.txt', sep=",", header=None)
    return df

In [2]:
def iter_minibatches(chunksize):
    chunkstartmarker = 0
    while chunkstartmarker < len(songs):
        x_chunk = songs_x_train[chunkstartmarker:chunkstartmarker+chunksize]
        y_chunk = songs_y_train[chunkstartmarker:chunkstartmarker+chunksize]
        yield x_chunk, y_chunk
        chunkstartmarker += chunksize

In [3]:
#songs é um DataFrame
songs = load_song_data()

In [4]:
songs_x = songs.iloc[:,1:]
songs_y = songs.iloc[:,0]
#songs_x = songs_x.values.reshape(-1,1)
songs_y = songs_y.values.reshape(-1,1)

In [5]:
#Normalizing
num_pipeline = Pipeline([
    ('std_scaler', StandardScaler()),
])
prepared = num_pipeline.fit_transform(songs_x)

In [6]:
songs_x_train, songs_x_test, songs_y_train, songs_y_test = train_test_split(prepared, songs_y, test_size=0.045, random_state=0)

In [7]:
for count, degree in enumerate([2]):
    batcherator = iter_minibatches(chunksize=1000)
    regr = make_pipeline(PolynomialFeatures(degree), SGDRegressor(learning_rate='invscaling', eta0=0.001))
    
    for x_chunk, y_chunk in batcherator:
        if(len(x_chunk)!=0):
            regr.named_steps['sgdregressor'].partial_fit(x_chunk, y_chunk)
    
songs_y_pred = regr.named_steps['sgdregressor'].predict(songs_x_test)

/home/fagner/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [8]:
# The coefficients
print('Estimated intercept: ', regr.named_steps['sgdregressor'].intercept_)

# The coefficients
print('Coefficients: ', regr.named_steps['sgdregressor'].coef_)

# The mean squared error
print("Mean squared error: %.2f"
      % np.sqrt(mean_squared_error(songs_y_test, songs_y_pred)))

Estimated intercept:  [ 1998.36661289]
Coefficients:  [  5.22521892e+00  -2.64670159e+00  -7.97856298e-01  -1.39416188e+00
  -2.50587275e-01  -2.23565230e+00   4.13016948e-01  -9.34815825e-01
  -8.27384110e-01   5.25684017e-01  -6.57380541e-02  -1.97507499e-01
   1.03790728e+00   6.28977793e-01  -4.81380468e-01   8.58197780e-01
   3.08793118e-01   1.03076891e+00   8.43921712e-01   1.07700909e+00
   2.50551084e-01  -4.41119085e-01   1.42183477e+00   3.89100291e-01
  -3.59286763e-01   4.27487473e-02   7.54274895e-01   6.80844827e-02
   2.65001581e-01   1.55355554e-01  -9.57659408e-02  -6.26350936e-02
  -4.58174806e-01   1.12100040e-01  -1.65665058e-02  -6.28047496e-01
  -4.28660724e-02   1.88870246e-01   5.41109943e-01  -3.27090392e-01
  -2.76203762e-01  -7.62275591e-02  -1.00449565e-01  -1.06868674e-01
  -9.51075084e-02   2.41133280e-01   1.77175011e-01  -5.90514844e-01
   3.82029799e-02   2.57605193e-01   2.29617739e-03  -1.55980147e-01
   1.63114379e-01  -9.84267556e-02   5.14565325e-